In [1]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [2]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [3]:
CONFIG_FILE_PATH = 'config/inD_longterm.yaml'  # yaml config file containing all the hyperparameters
DATASET_NAME = 'ind'

TEST_DATA_PATH = 'data/inD/test.pkl'
TEST_IMAGE_PATH = 'data/inD/test'
OBS_LEN = 5  # in timesteps
PRED_LEN = 30  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a

ROUNDS = 3  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8

#### Load config file and print hyperparameters

In [4]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'resize': 0.33,
 'batch_size': 8,
 'viz_epoch': 10,
 'learning_rate': 0.0001,
 'num_epochs': 300,
 'encoder_channels': [32, 32, 64, 64, 64],
 'decoder_channels': [64, 64, 64, 32, 32],
 'waypoints': [14, 29],
 'temperature': 1.8,
 'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
 'semantic_classes': 6,
 'loss_scale': 1000,
 'kernlen': 31,
 'nsig': 4,
 'use_features_only': False,
 'unfreeze': 100,
 'use_TTST': True,
 'rel_threshold': 0.002,
 'use_CWS': True,
 'CWS_params': {'sigma_factor': 6, 'ratio': 2, 'rot': True}}

#### Load preprocessed Data

In [5]:
df_test = pd.read_pickle(TEST_DATA_PATH)

In [6]:
df_test.head()

,trackId,frame,x,y,sceneId,metaId,frame_diff,recId
0,98,6852,854.241010,141.705840,scene1,0,1.0,00
1,98,6877,853.749934,149.903018,scene1,0,1.0,00
2,98,6902,852.634186,158.528806,scene1,0,1.0,00
3,98,6927,851.607283,167.000131,scene1,0,1.0,00
4,98,6952,850.816207,175.933858,scene1,0,1.0,00


#### Initiate model and load pretrained weights

In [7]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

In [8]:
model.load(f'pretrained_models/{experiment_name}_weights.pt')

<All keys matched successfully>


#### Evaluate model

In [9]:
model.evaluate(df_test, params, image_path=TEST_IMAGE_PATH,
               batch_size=BATCH_SIZE, rounds=ROUNDS, 
               num_goals=NUM_GOALS, num_traj=NUM_TRAJ, device=None, dataset_name=DATASET_NAME)

Round:   0%|                                                                                     | 0/3 [00:00<?, ?it/s]

Preprocess data
Start testing


Round:  33%|█████████████████████████▎                                                  | 1/3 [03:09<06:18, 189.02s/it]

Round 0: 
Test ADE: 14.850406646728516 
Test FDE: 19.687257766723633


Round:  67%|██████████████████████████████████████████████████▋                         | 2/3 [06:16<03:08, 188.58s/it]

Round 1: 
Test ADE: 14.665905952453613 
Test FDE: 19.303884506225586


Round: 100%|████████████████████████████████████████████████████████████████████████████| 3/3 [09:24<00:00, 188.20s/it]

Round 2: 
Test ADE: 14.900250434875488 
Test FDE: 19.748842239379883


Average performance over 3 rounds: 
Test ADE: 14.805521011352539 
Test FDE: 19.579994837443035
